In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('results.csv')

In [ ]:
print(dataset.head())

       exam1      exam2  admission
0  34.623660  78.024693          0
1  30.286711  43.894998          0
2  35.847409  72.902198          0
3  60.182599  86.308552          1
4  79.032736  75.344376          1


In [ ]:
shuffled_dataset = dataset.sample(frac=1)
X = dataset.iloc[:,:-1]
Y = dataset.iloc[:,-1]

In [ ]:
def split_data(fraction_size, dataset):
    train_size = int(fraction_size*dataset.shape[0])
    train = dataset.iloc[:train_size,:]
    test  = dataset.iloc[train_size:,:]
    X_test = test.iloc[:,:-1].values
    Y_test = test.iloc[:,-1].values
    return train, test, X_test,Y_test

In [ ]:
def gradient_batch(X,Y,L=0.001):
    m = np.zeros(shape=(1,X.shape[1]))
    c = 0
    epochs = 2000
    theta0=[]
    theta1=[]
    cost = []
    # epochs value choose inversely proportional to L
    n = float(len(Y))
    sample = X.shape[0]
    for iter in range(epochs):
        temp_m=  np.zeros(shape=(1,X.shape[1]))
        temp_c = 0
        for i in range(len(Y)):
            Y_pred = np.dot(m,X[i])+c
            temp_m =  X[i]*((Y[i]- Y_pred))
            temp_c =  (Y[i]- Y_pred)
        m = m - L*(-2/sample)*temp_m
        c = c - L*(-2/sample)*temp_c
    return m,c

In [ ]:
def predict(X,m,c):
    y = []
    for i in range(len(X)):
        prediction = np.dot(m,X[i])+c
        y.append(prediction)
    return y

In [ ]:
def calc_MAE(Y_predict, Y):
    mae = 0
    n = len(Y)
    for i in range (n):
        mae += abs((Y_predict[i]-Y[i]))
    mae = (mae*100)/n
    return mae

# Batch GD

In [ ]:
def batch_GD(train_data,learning_rate,decL, epoch):
    w=np.ones(shape=(1,train_data.shape[1]-1))
    b=0
    cur_iter=1
    while(cur_iter<=epoch): 
        temp=train_data
        y=np.array(temp['admission'])
        x=np.array(temp.drop('admission',axis=1))
        w_gradient=np.zeros(shape=(1,train_data.shape[1]-1))
        b_gradient=0
        n = train_data.shape[0]
        for i in range(n):
            prediction=np.dot(w,x[i])+b
            w_gradient=w_gradient+(2)*x[i]*(y[i]-(prediction))
            b_gradient=b_gradient+(2)*(y[i]-(prediction))
        w=w-learning_rate*(w_gradient/n)
        b=b-learning_rate*(b_gradient/n)
        
        cur_iter=cur_iter+1
#         learning_rate=learning_rate/decL
    return w,b

# Stochastic GD

In [ ]:
def stochastic_GD(train_data,learning_rate,decL, epoch):
    w=np.zeros(shape=(1,train_data.shape[1]-1))
    b=0
    cur_iter=1
    while(cur_iter<=epoch): 
        temp=train_data
        y=np.array(temp['admission'])
        x=np.array(temp.drop('admission',axis=1))
        w_gradient=np.zeros(shape=(1,train_data.shape[1]-1))
        b_gradient=0
        i = cur_iter % len(y)
        prediction=np.dot(w,x[i])+b
        w_gradient=w_gradient+(-2)*x[i]*(y[i]-(prediction))
        b_gradient=b_gradient+(-2)*(y[i]-(prediction))
        w=w-learning_rate*(w_gradient)
        b=b-learning_rate*(b_gradient)
        
        cur_iter=cur_iter+1
#         learning_rate=learning_rate/decL
    return w,b

# Mini Batch GD 
### default size = 50

In [ ]:
def miniBatch_GD(train_data,learning_rate,k,decL, epoch):
    w=np.zeros(shape=(1,train_data.shape[1]-1))
    b=0
    cur_iter=1
    start = -k
    while(cur_iter<=epoch): 
        temp=train_data
        
        y=np.array(temp['admission'])
        x=np.array(temp.drop('admission',axis=1))
        w_gradient=np.zeros(shape=(1,train_data.shape[1]-1))
        b_gradient=0
        start = (start + k) %len(y)
        for i in range(start, start + k):
            i = i % len(y)
            prediction=np.dot(w,x[i])+b
            w_gradient=learning_rate*w_gradient+(-2)*learning_rate*x[i]*(y[i]-(prediction))
            b_gradient=learning_rate*b_gradient+(-2)*learning_rate*(y[i]-(prediction))
        w=w-(w_gradient/k)
        b=b-(b_gradient/k)
        cur_iter=cur_iter+1
#         learning_rate=learning_rate/decL
    return w,b

# Splitting Data

In [ ]:
train, test, X_test, Y_test = split_data(0.7, dataset)



# Without Feature Scaling


In [ ]:
y  = train.iloc[:,1].values
tr = train
tr = tr.drop('admission',axis=1)
x  = tr.values 
m1,c1 = gradient_batch(x,y,L=0.01)
Y_predict = predict(X_test,m1[0],c1[0])

for i in range(len(Y_predict)):
    if(Y_predict[i] > 0.5):
        Y_predict[i] = 1
    else:
        Y_predict[i] = 0
# for i in range(len(X_test)):
#     print(Y_test[i], Y_predict[i])
mae1 = calc_MAE(Y_predict, Y_test)
print("Mean Absolute Error in Test  Data = {} %".format(mae1))

Mean Absolute Error in Test  Data = 80.0 %


In [ ]:
m2,c2 = stochastic_GD(train.iloc[:,:], 0.00001, 2, 3000)
Y_predict2 = predict(X_test,m2[0],c2[0])
# print(m2,c2)

for i in range(len(Y_predict2)):
    if(Y_predict2[i] > 0.5):
        Y_predict2[i] = 1
    else:
        Y_predict2[i] = 0
# for i in range(len(X_test)):
#     print(Y_test[i], Y_predict2[i])
mae2 = calc_MAE(Y_predict2, Y_test)
print("Mean Absolute Error in Test  Data = {} %".format(mae2))

Mean Absolute Error in Test  Data = 16.666666666666668 %


In [ ]:
m3,c3 = miniBatch_GD(train.iloc[:,:], 0.0000001,  30, 2, 2000)
Y_predict3 = predict(X_test,m3[0],c3[0])

for i in range(len(Y_predict3)):
    if(Y_predict3[i] > 0.5):
        Y_predict3[i] = 1
    else:
        Y_predict3[i] = 0

mae3 = calc_MAE(Y_predict3, Y_test)
print("Mean Absolute Error in Test  Data = {} %".format(mae3))

Mean Absolute Error in Test  Data = 80.0 %


# Feature Scaling

In [ ]:
def standardize(data, col):
    temp = np.array(data[col])
    sd = np.std(temp)
    mean = np.mean(temp)
    temp = (temp - mean) / sd
    data[col] = temp
    return data

In [ ]:
# dataset = standardize(dataset,'exam2')
dataset = standardize(dataset,'exam1')
train, test, X_test, Y_test = split_data(0.7, dataset)

In [ ]:
y  = train.iloc[:,1].values
tr = train
tr = tr.drop('admission',axis=1)
x  = tr.values 
m21,c21 = gradient_batch(x,y,L=0.00000004)
Y_predict21 = predict(X_test,m21[0],c21[0])
# print(tr)
# for i in range(len(X_test)):
#     print(Y_test[i], Y_predict21[i])
for i in range(len(Y_predict21)):
    if(Y_predict21[i] > 0.5):
        Y_predict21[i] = 1
    else:
        Y_predict21[i] = 0
  
mae21 = calc_MAE(Y_predict21, Y_test)

print("Mean Absolute Error in Test  Data = {} %".format(mae21))


Mean Absolute Error in Test  Data = 16.666666666666668 %


In [ ]:
m22,c22 = stochastic_GD(train.iloc[:,:], 0.0001, 2, 5000)
Y_predict22 = predict(X_test,m22[0],c22[0])
for i in range(len(Y_predict22)):
    if(Y_predict22[i] > 0.5):
        Y_predict22[i] = 1
    else:
        Y_predict22[i] = 0
mae22 = calc_MAE(Y_predict22, Y_test)
print("Mean Absolute Error in Test  Data = {} %".format(mae22))

Mean Absolute Error in Test  Data = 13.333333333333334 %


In [ ]:
m23,c23 = miniBatch_GD(train.iloc[:,:], 0.000001,  20, 2, 5000)
Y_predict23 = predict(X_test,m23[0],c23[0])
for i in range(len(Y_predict21)):
    if(Y_predict23[i] > 0.5):
        Y_predict23[i] = 1
    else:
        Y_predict23[i] = 0
mae23 = calc_MAE(Y_predict23, Y_test)
print("Mean Absolute Error in Test  Data = {} %".format(mae23))

Mean Absolute Error in Test  Data = 16.666666666666668 %


# Compare

In [ ]:
print("Without Feature Scaling-----")
print("===========================================")
print("Batch GD: MAEP      = {} %".format(mae1))
print("Stochastic GD: MAEP = {} %".format(mae2))
print("Mini-Batch GD: MAEP = {} %".format(mae3))
print()
print("Feature Scaling using standardization------")
print("===========================================")
print("Batch GD: MAEP      = {} %".format(mae21))
print("Stochastic GD: MAEP = {} %".format(mae22))
print("Mini-Batch GD: MAEP = {} %".format(mae23))

Without Feature Scaling-----
Batch GD: MAEP      = 80.0 %
Stochastic GD: MAEP = 16.666666666666668 %
Mini-Batch GD: MAEP = 80.0 %

Feature Scaling using standardization------
Batch GD: MAEP      = 16.666666666666668 %
Stochastic GD: MAEP = 13.333333333333334 %
Mini-Batch GD: MAEP = 16.666666666666668 %
